# Importing Necessary Modules

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Conv2D, Flatten, MaxPooling2D, AveragePooling2D
from tensorflow.keras.applications import vgg16

# Showing Vgg16 architecture

In [ ]:
vgg16_original_model = vgg16.VGG16()
vgg16_original_model.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

# Copying vgg16 architecture

In [ ]:
#input Layer
inputs = Input((224,224, 3), name='input')

#Block1
x  = Conv2D(filters=64, kernel_size=(3,3), name='block1_conv1', padding='same', activation='relu')(inputs)
x  = Conv2D(filters=64, kernel_size=(3,3), name='block1_conv2', padding='same', activation='relu')(x)
x = MaxPooling2D(name = 'block1_pool')(x)

#Block2
x  = Conv2D(filters=128, kernel_size=(3,3), name='block2_conv1', padding='same', activation='relu')(x)
x  = Conv2D(filters=128, kernel_size=(3,3), name='block2_conv2', padding='same', activation='relu')(x)
x = MaxPooling2D(name = 'block2_pool')(x)


#Block3
x  = Conv2D(filters=256, kernel_size=(3,3), name='block3_conv1', padding='same', activation='relu')(x)
x  = Conv2D(filters=256, kernel_size=(3,3), name='block3_conv2', padding='same', activation='relu')(x)
x  = Conv2D(filters=256, kernel_size=(3,3), name='block3_conv3', padding='same', activation='relu')(x)
x = MaxPooling2D(name = 'block3_pool')(x)


#Block4
x  = Conv2D(filters=512, kernel_size=(3,3), name='block4_conv1', padding='same', activation='relu')(x)
x  = Conv2D(filters=512, kernel_size=(3,3), name='block4_conv2', padding='same', activation='relu')(x)
x  = Conv2D(filters=512, kernel_size=(3,3), name='block4_conv3', padding='same', activation='relu')(x)
x = MaxPooling2D(name = 'block4_pool')(x)


#Block5
x  = Conv2D(filters=512, kernel_size=(3,3), name='block5_conv1', padding='same', activation='relu')(x)
x  = Conv2D(filters=512, kernel_size=(3,3), name='block5_conv2', padding='same', activation='relu')(x)
x  = Conv2D(filters=512, kernel_size=(3,3), name='block5_conv3', padding='same', activation='relu')(x)
x = MaxPooling2D(name = 'block5_pool')(x)


x = Flatten()(x)
#Dense
x = Dense(4096, name='fc1', activation='relu')(x)
x = Dense(4096, name='fc2', activation='relu')(x)

outputs = Dense(1000, activation = 'sigmoid', name='predictions')(x)
vgg_copy = Model(inputs, outputs, name='vgg16_copy')
vgg_copy.summary()

Model: "vgg16_copy"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0

# Transfer weights of pre-trained VGG16 to the newly built VGG16

In [ ]:
for i in range(len(vgg_copy.layers)):
    vgg_copy.layers[i].set_weights(vgg16_original_model.layers[i].get_weights())

# A CNN which will look like VGG16, but will not be exactly VGG16

In [ ]:
# Input layer with a different shape
inputs_custom = Input((128, 128, 3), name='input_custom')

# Block 1 with a different number of convolution layers
x_custom = Conv2D(32, (3, 3), activation='relu', padding='same', name='block1_conv1_custom')(inputs_custom)
x_custom = Conv2D(32, (3, 3), activation='relu', padding='same', name='block1_conv2_custom')(x_custom)
x_custom = MaxPooling2D((2, 2), name='block1_pool_custom')(x_custom)

# Block 2 with a different number of neurons in dense layers
x_custom = Conv2D(64, (3, 3), activation='relu', padding='same', name='block2_conv1_custom')(x_custom)
x_custom = Conv2D(64, (3, 3), activation='relu', padding='same', name='block2_conv2_custom')(x_custom)
x_custom = MaxPooling2D((2, 2), name='block2_pool_custom')(x_custom)

# Block 3 with a different down-sampling technique (AveragePooling2D)
x_custom = Conv2D(128, (3, 3), activation='relu', padding='same', name='block3_conv1_custom')(x_custom)
x_custom = Conv2D(128, (3, 3), activation='relu', padding='same', name='block3_conv2_custom')(x_custom)
x_custom = Conv2D(128, (3, 3), activation='relu', padding='same', name='block3_conv3_custom')(x_custom)
x_custom = AveragePooling2D((2, 2), name='block3_avg_pool_custom')(x_custom)

# Flatten and different number of dense layers
x_custom = Flatten(name='flatten_custom')(x_custom)
x_custom = Dense(512, activation='relu', name='fc1_custom')(x_custom)
x_custom = Dense(256, activation='relu', name='fc2_custom')(x_custom)

# Different output shape and activation function
outputs_custom = Dense(10, activation='softmax', name='predictions_custom')(x_custom)

# Custom CNN model
custom_cnn = Model(inputs_custom, outputs_custom, name='custom_cnn')
custom_cnn.summary()


Model: "custom_cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_custom (InputLayer)   [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1_custom (Conv2  (None, 128, 128, 32)      896       
 D)                                                              
                                                                 
 block1_conv2_custom (Conv2  (None, 128, 128, 32)      9248      
 D)                                                              
                                                                 
 block1_pool_custom (MaxPoo  (None, 64, 64, 32)        0         
 ling2D)                                                         
                                                                 
 block2_conv1_custom (Conv2  (None, 64, 64, 64)        18496     
 D)                                                     